# Modules

In [1]:
import pandas as pd
from io import StringIO
import numpy as np
import re

# Variables

In [2]:
input_folder = '../data/raw_data/popdata'
output_folder = '../data/processed_data/popdata'

# Process data

In [3]:
year = 2018

## Population data

In [7]:
df_raw_dane_data = pd.read_excel(f'{input_folder}/Edades_Simples_1985-2020.xls', sheet_name='Total')

df_raw_dane_data.to_csv('tmp_raw_dane_data', index=False)
with open('tmp_raw_dane_data', 'r') as file:
    data = file.readlines()

In [8]:
line_increment = 19
init_line = 11
pivot_line = init_line

years     = [line.strip().split(',') for line in data[init_line-2 : init_line - 1]]
years     = np.sort(np.unique(years))
correct_column_name = np.append(['Codigo', 'Grupos de edad'], np.array([(f'Total_{year}', f'Hombres_{year}', f'Mujeres_{year}') for year in years[years!='']]).flatten())

pattern = r'^([^,]+),'
df_pop = pd.DataFrame({})
while True:
    match  = re.search(pattern, data[pivot_line])
    codigo = match.group(1)
    ##print(codigo)
    if 'DANE' in codigo:
        break

    data_rows = [line.strip().split(',') for line in data[pivot_line   : pivot_line + line_increment]]
    columns   = [line.strip().split(',') for line in data[pivot_line-1 : pivot_line]]

    df_ = pd.DataFrame(data_rows[1:], columns=correct_column_name).melt(id_vars=["Codigo", "Grupos de edad"]).rename(columns={'Grupos de edad':'AgeGroup',                                                                                                                    
                                                                                                                              'Codigo':'Code',
                                                                                                                              'value':'Pop'})
    df_['Year']     = df_.variable.str[-4:]
    df_['Gender']   = df_.variable.str[:-5].replace('Mujeres', 'Female').replace('Hombres', 'Male')
    df_['AgeGroup'] = df_['AgeGroup'].replace('80 Y MÁS', '80-above')
    df_['Code']     = codigo[:2]
    df_['Zone']     = codigo
    del df_['variable']

    df_pop = pd.concat([df_pop, df_]).reset_index(drop=True)

    pivot_line += line_increment

In [10]:
df_pop = df_pop[df_pop['AgeGroup'] != 'Total']
df_pop[['MinAge', 'MaxAge']] = df_pop['AgeGroup'].str.split('-', expand=True)
df_pop['MaxAge'] = df_pop['MaxAge'].replace('above', '200')

In [11]:
df_pop.to_csv(f'{output_folder}/colombia_population_data_municp.csv', index=False)

## Household data

In [64]:
with open(f'{input_folder}/Personas_por_hogar.csv', 'r') as file:
    lines = file.readlines()

In [65]:
pattern_code = r'AREA # (\d+)'
pattern_name = r',([A-ZÑ\s]+|"[A-ZÑ\s,]+")\s*-?\s*([A-ZÑ\s]+|"[A-ZÑ\s,]+"),'

In [66]:
# Identify the start and end points of the tables
start_indices  = [i for i, line in enumerate(lines) if 'AREA #' in line]
end_indices    = [i for i, line in enumerate(lines) if 'Total' in line]
codigos_municp = [re.search(pattern_code, line).group(1) for i, line in enumerate(lines) if 'AREA #' in line]
codigos_depart = [code_municp[:2] for code_municp in codigos_municp]
names_depart   = [re.search(pattern_name, line.replace('"','').replace(' - ',' ')).group(1) for i, line in enumerate(lines) if 'AREA #' in line]
names_municp   = [re.search(pattern_name, line.replace('"','').replace(' - ',' ')).group(2) for i, line in enumerate(lines) if 'AREA #' in line]

dataframes = []

# Extract tables and read into dataframes
for start, end, code_depart, code_municp, name_depart, name_municp in zip(start_indices, end_indices, codigos_depart, codigos_municp, names_depart, names_municp):
    table_str = ''.join(lines[start:end]).strip()
    # Check if there's content to parse
    if 'Personas por Hogar' in table_str:
        table_start = table_str.index('Personas por Hogar')
        df = (pd.read_csv(StringIO(table_str[table_start:]), sep=',')
                                                                    .reset_index(drop=True)
                                                                    .rename(columns={'Personas por Hogar':'PersonsHousehold',
                                                                                     'Casos':'NumHouses'}))[['PersonsHousehold', 'NumHouses']]
        df['Year'] = year
        df['Code'] = code_depart
        df['Zone'] = code_municp
        df['Departamento']    = name_depart
        df['Municipio']       = name_municp
        dataframes.append(df)

In [67]:
df_household = pd.concat(dataframes)[["Code","Zone","NumHouses","PersonsHousehold","Year", 'Departamento', 'Municipio']]
df_household['NumHouses'] = df_household['NumHouses'].astype(str).str.replace(' ','',regex=True).astype(int)

In [70]:
df_household.to_csv(f'{output_folder}/colombia_household_composition_municp.csv', index=False)